In [ ]:
# This is the consequence processor for sgRNA designed by CHOP CHOP
# This notebook will process the local chopchop output and is tied to the source function sgFinder
# Author: Yiyun
# Date: Sep, 2022

In [68]:
# from os import listdir
# from os.path import join
# import itertools
# import pandas as pd 
# from tqdm import tqdm
# import pickle
# from Bio import SeqIO

from os.path import exists
from os import listdir,makedirs

import sys
sys.path.append('../')
from src import syn_sgFinder

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
out_path = '../data/output/sg_Finder'
mrna_path = '../data/MANE'
fpath = '../data/output/sg_out/'

# Load all expressed transcripts
f_genelist = pd.read_csv('../data/output/gene_list.txt', header =None)
list_exp_trsp = pickle.load(open('../data/22Q2_gene_effect/expressed_transcripts.pkl','rb'))
# Parse mRNA sequence dictionary, for use when sequence aligned to intron-exon junctions
record_dict = SeqIO.to_dict(SeqIO.parse(join(mrna_path,'GRCh38_latest_rna.fna'), "fasta"))

In [76]:
def run_sgFinder(input_genes, tag = 'ess', sg_p ='../data/output/sg_out' ):
    out_path = '../data/output/sg_Finder'
    
    df_synsg_abe = pd.DataFrame()
    df_synsg_cbe = pd.DataFrame()
    df_abeCT = pd.DataFrame()
    df_cbeCT = pd.DataFrame()

    for genes in tqdm(input_genes):
        sg_gene = syn_sgFinder.sgFinder_gene(genes, sg_path = sg_p)
        try:
            sg_gene.gene_cds_proc()
        except ValueError as e:
            print(e)
            continue

        df_synsg_abe_dt = pd.DataFrame()
        df_synsg_cbe_dt = pd.DataFrame()
        for idx, transcripts in list(itertools.product(sg_gene.sg_file.index,\
                                                           sg_gene.dict_exons.keys())):
            if not transcripts in list_exp_trsp:
    #             print(f'{transcripts} not expressed, skipping...')
                continue # Pass the transcripts that are not expressed
            else:
                sg_sg = syn_sgFinder.sgFinder_sg(sg_gene.gene,idx)
                check_point = sg_sg.process_BE_window(sg_gene)
                if check_point:
                    sg_trsp = syn_sgFinder.sgFinder_transcript(sg_gene.gene,idx,transcripts)
                    sg_trsp.all_pos_consq(sg_gene,sg_sg,record_dict)

                    df_synsg_abe = df_synsg_abe.append(sg_trsp.df_abe,ignore_index = True)
                    df_synsg_cbe = df_synsg_cbe.append(sg_trsp.df_cbe,ignore_index = True)

                    df_abeCT = df_abeCT.append(sg_trsp.df_abe_control,ignore_index = True)
                    df_cbeCT = df_cbeCT.append(sg_trsp.df_cbe_control,ignore_index = True)

                    df_synsg_abe_dt = df_synsg_abe_dt.append(sg_trsp.df_abe_ind,ignore_index = True)
                    df_synsg_cbe_dt = df_synsg_cbe_dt.append(sg_trsp.df_cbe_ind,ignore_index = True)

        df_synsg_abe_dt.to_csv(join(out_path,genes,f'df_abe_detail.csv'))
        df_synsg_cbe_dt.to_csv(join(out_path,genes,f'df_cbe_detail.csv'))
    
    # Process and saving final result
    ABE_synsg = df_synsg_abe[df_synsg_abe['syn_or_not'].apply(lambda x: \
                                                              all([i=='True' for i in x.split("[")[1].split(']')[0].split(',')]))]
    CBE_synsg= df_synsg_cbe[df_synsg_cbe['syn_or_not'].apply(lambda x: \
                                                  all([i=='True' for i in x.split("[")[1].split(']')[0].split(',')]))]
    
    if not exists(join(out_path,tag)):
        makedirs(join(out_path,tag))
    ABE_synsg.to_csv(join(out_path,tag,'ABE_synsg.csv'))
    CBE_synsg.to_csv(join(out_path,tag,'CBE_synsg.csv'))

    df_abeCT.to_csv(join(out_path,tag,'ABE_CT.csv'))
    df_cbeCT.to_csv(join(out_path,tag,'CBE_CT.csv'))

***Run essential + depletion gene list***

In [74]:
fpath = '../data/output/sg_out/'
list_gene = [i for i in listdir(fpath) if not i.startswith('.')]
list_gene = [i for i in list_gene if len(listdir(join(fpath,i))) > 5]
list_gene = [i for i in list_gene if i in list(f_genelist[0])]
run_sgFinder(list_gene, tag = 'ess' )

  0%|                                                    | 0/92 [00:00<?, ?it/s]

Processing UBE2S...Keep 6/331 low off-target sgRNAs...
Processing RRS1...Keep 501/587 low off-target sgRNAs...


  2%|▉                                           | 2/92 [00:05<03:57,  2.64s/it]

Processing PBX2...Keep 0/555 low off-target sgRNAs...
Processing CSPG5...Keep 395/492 low off-target sgRNAs...
47577601 is in start/stop codon ...skippingATGCCCCCTGC
ATGCCCCCTGCCATTCCTGA in gene CSPG5 not aligned, skip...


  4%|█▉                                          | 4/92 [00:10<04:03,  2.77s/it]

...

Processing DUT...Keep 55/125 low off-target sgRNAs...
CTGTACAGGTCGTAGCCCGC in gene DUT not aligned, skip...
GCCTTGGATGACACCGAAAG in gene DUT not aligned, skip...


 98%|██████████████████████████████████████████ | 90/92 [05:58<00:07,  3.55s/it]

Processing COA3...Keep 122/142 low off-target sgRNAs...
42798671 is in start/stop codon ...skippingATGGCGTCTTC
ATGGCGTCTTCGGGAGCTGG in gene COA3 not aligned, skip...
42798073 is in start/stop codon ...skippingAGGACCCTGAC
Finding mrna sequence in another exon...
Finding extra sequence in mRNA failed, skip...
Finding mrna sequence in another exon...
Finding extra sequence in mRNA failed, skip...


100%|███████████████████████████████████████████| 92/92 [06:00<00:00,  3.91s/it]

Processing FLOT1...Keep 0/430 low off-target sgRNAs...


***Run opposite + depletion gene list***

In [77]:
list_gene_enrich = [i for i in listdir('../data/output/sg_out_enrich') if not i.startswith('.')]
list_gene_enrich = [i for i in list_gene_enrich if len(listdir(join('../data/output/sg_out_enrich',i))) > 5]
run_sgFinder(list_gene_enrich, tag = 'enr' , sg_p ='../data/output/sg_out_enrich' )

  6%|██▌                                         | 1/17 [00:00<00:02,  5.99it/s]

Processing AMN...Keep 574/648 low off-target sgRNAs...
Processing NKX6-1...Keep 436/601 low off-target sgRNAs...


 12%|█████▏                                      | 2/17 [00:00<00:02,  6.27it/s]

Processing ZBTB20...Keep 905/1017 low off-target sgRNAs...


 24%|██████████▎                                 | 4/17 [00:13<00:48,  3.75s/it]

...

 82%|███████████████████████████████████▍       | 14/17 [00:32<00:03,  1.26s/it]

Processing AOAH...Keep 423/494 low off-target sgRNAs...


 94%|████████████████████████████████████████▍  | 16/17 [00:33<00:00,  1.35it/s]

Processing HOXD1...Keep 450/527 low off-target sgRNAs...
Processing C2CD4B...Keep 294/673 low off-target sgRNAs...


100%|███████████████████████████████████████████| 17/17 [00:33<00:00,  1.96s/it]


***For Debugging...***
***Single gene checks***

In [49]:
sg_gene = syn_sgFinder.sgFinder_gene('MCL1')
sg_gene.gene_cds_proc()
sg_sg = syn_sgFinder.sgFinder_sg(sg_gene.gene,2)
sg_sg.process_BE_window(sg_gene)
sg_trsp = syn_sgFinder.sgFinder_transcript(sg_gene.gene,2, 'NM_021960')
sg_trsp.all_pos_consq(sg_gene,sg_sg)

Processing MCL1...Keep 116/144 low off-target sgRNAs...
150579530
[150579499, 2, Seq('TCCGATTACCGCGTTTCTTT'), (150579500, 150579498), (2, 4), Seq('GGA'), Seq('GAA')]
150579530
[150579500, 1, Seq('TCCGATTACCGCGTTTCTTT'), (150579500, 150579498), (2, 4), Seq('GGA'), Seq('AGA')]
150579530
[150579506, 1, Seq('TCCGATTACCGCGTTTCTTT'), (150579506, 150579504), (8, 10), Seq('GTA'), Seq('ATA')]
150579530
[150579507, 3, Seq('TCCGATTACCGCGTTTCTTT'), (150579509, 150579507), (11, 13), Seq('GCG'), Seq('GCA')]
150579530
[150579502, 2, Seq('TCCGATTACCGCGTTTCTTT'), (150579503, 150579501), (5, 7), Seq('ATC'), Seq('ACC')]
150579530
[150579505, 2, Seq('TCCGATTACCGCGTTTCTTT'), (150579506, 150579504), (8, 10), Seq('GTA'), Seq('GCA')]
